# Questionnaire Data

In [22]:
import pandas as pd
import numpy as np

import sklearn
import os

import scipy
import neurokit2 as nk
from scipy.stats import pearsonr

import tqdm 

from collections import Counter
import matplotlib.pyplot as plt

from pathlib import Path
Path("./figures").mkdir(parents=True, exist_ok=True)

In [23]:
pathQ = "./QData/"

dfQAll1 = pd.read_csv(f"{pathQ}QData_Clean.csv")
dfQAll1

,PId,BlockNumber,Age,Gender,NASAMentalDemand,NASAPhysDemand,NASATempDemand,NASAPerformance,NASAEffort,NASAFrustration,UX1,UX2,GEQ_Competence1,GEQ_Competence2,GEQ_Immersion1,GEQ_Immersion2,GEQ_PosAff1,GEQ_PosAff2
0,1,1,23,male,5,1,11,4,4,5,4,4,0,2,0,1,1,1
1,1,6,23,male,5,1,6,2,8,5,4,2,0,2,0,0,1,2
2,1,4,23,male,10,1,12,3,9,2,4,4,2,2,0,0,1,1
3,1,2,23,male,6,1,7,2,4,4,4,6,3,1,0,0,1,2
4,1,7,23,male,3,1,14,1,2,1,4,2,3,4,0,0,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,20,7,26,male,10,13,6,4,11,13,5,5,3,3,2,3,3,3
115,20,3,26,male,10,13,9,3,10,16,3,5,4,2,2,2,3,2
116,20,2,26,male,10,16,6,4,10,17,3,5,4,3,2,2,3,3
117,20,5,26,male,10,13,6,5,10,18,5,5,4,3,1,2,3,2


In [24]:
dfNASA = dfQAll1[['PId','BlockNumber','NASAMentalDemand', 'NASAPhysDemand','NASATempDemand', 'NASAPerformance', 'NASAEffort', 'NASAFrustration']]
dfNASA


dfNASA['RawNASASum'] = dfNASA[['NASAMentalDemand', 'NASAPhysDemand','NASATempDemand', 'NASAPerformance', 'NASAEffort', 'NASAFrustration']].sum(axis=1)
dfNASA["RawNASA"] = dfNASA.RawNASASum /120*2
#del dfNASA["RawNASASum"]

dfNASA

C:\Users\ru74yig\AppData\Local\Temp\ipykernel_9396\2743240409.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNASA['RawNASASum'] = dfNASA[['NASAMentalDemand', 'NASAPhysDemand','NASATempDemand', 'NASAPerformance', 'NASAEffort', 'NASAFrustration']].sum(axis=1)
C:\Users\ru74yig\AppData\Local\Temp\ipykernel_9396\2743240409.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNASA["RawNASA"] = dfNASA.RawNASASum /120*2


,PId,BlockNumber,NASAMentalDemand,NASAPhysDemand,NASATempDemand,NASAPerformance,NASAEffort,NASAFrustration,RawNASASum,RawNASA
0,1,1,5,1,11,4,4,5,30,0.500000
1,1,6,5,1,6,2,8,5,27,0.450000
2,1,4,10,1,12,3,9,2,37,0.616667
3,1,2,6,1,7,2,4,4,24,0.400000
4,1,7,3,1,14,1,2,1,22,0.366667
...,...,...,...,...,...,...,...,...,...,...
114,20,7,10,13,6,4,11,13,57,0.950000
115,20,3,10,13,9,3,10,16,61,1.016667
116,20,2,10,16,6,4,10,17,63,1.050000
117,20,5,10,13,6,5,10,18,62,1.033333


In [25]:
dfGEQ = dfQAll1[['PId','BlockNumber','GEQ_Competence1','GEQ_Competence2','GEQ_Immersion1','GEQ_Immersion2','GEQ_PosAff1','GEQ_PosAff2']]


dfGEQ


dfQAll = pd.merge(dfGEQ, dfNASA, on =["PId", "BlockNumber"], how="left")
dfQAll

,PId,BlockNumber,GEQ_Competence1,GEQ_Competence2,GEQ_Immersion1,GEQ_Immersion2,GEQ_PosAff1,GEQ_PosAff2,NASAMentalDemand,NASAPhysDemand,NASATempDemand,NASAPerformance,NASAEffort,NASAFrustration,RawNASASum,RawNASA
0,1,1,0,2,0,1,1,1,5,1,11,4,4,5,30,0.500000
1,1,6,0,2,0,0,1,2,5,1,6,2,8,5,27,0.450000
2,1,4,2,2,0,0,1,1,10,1,12,3,9,2,37,0.616667
3,1,2,3,1,0,0,1,2,6,1,7,2,4,4,24,0.400000
4,1,7,3,4,0,0,3,3,3,1,14,1,2,1,22,0.366667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,20,7,3,3,2,3,3,3,10,13,6,4,11,13,57,0.950000
115,20,3,4,2,2,2,3,2,10,13,9,3,10,16,61,1.016667
116,20,2,4,3,2,2,3,3,10,16,6,4,10,17,63,1.050000
117,20,5,4,3,1,2,3,2,10,13,6,5,10,18,62,1.033333


In [26]:
dfUX = dfQAll1[['PId','BlockNumber','UX1','UX2']]
dfUX

dfQAll = pd.merge(dfQAll, dfUX, on =["PId", "BlockNumber"], how="left")
dfQAll

,PId,BlockNumber,GEQ_Competence1,GEQ_Competence2,GEQ_Immersion1,GEQ_Immersion2,GEQ_PosAff1,GEQ_PosAff2,NASAMentalDemand,NASAPhysDemand,NASATempDemand,NASAPerformance,NASAEffort,NASAFrustration,RawNASASum,RawNASA,UX1,UX2
0,1,1,0,2,0,1,1,1,5,1,11,4,4,5,30,0.500000,4,4
1,1,6,0,2,0,0,1,2,5,1,6,2,8,5,27,0.450000,4,2
2,1,4,2,2,0,0,1,1,10,1,12,3,9,2,37,0.616667,4,4
3,1,2,3,1,0,0,1,2,6,1,7,2,4,4,24,0.400000,4,6
4,1,7,3,4,0,0,3,3,3,1,14,1,2,1,22,0.366667,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,20,7,3,3,2,3,3,3,10,13,6,4,11,13,57,0.950000,5,5
115,20,3,4,2,2,2,3,2,10,13,9,3,10,16,61,1.016667,3,5
116,20,2,4,3,2,2,3,3,10,16,6,4,10,17,63,1.050000,3,5
117,20,5,4,3,1,2,3,2,10,13,6,5,10,18,62,1.033333,5,5


In [28]:
dfQAll.to_csv("Q_ada.csv", index=False)


In [44]:
pathQ = "./QData/"

dfQAll2 = pd.read_csv(f"{pathQ}Q_data_table.csv")
dfQAll2

,PId,BlockNumber,GEQ_Competence1,GEQ_Competence2,GEQ_Immersion1,GEQ_Immersion2,GEQ_PosAff1,GEQ_PosAff2,NASAMentalDemand,NASAPhysDemand,...,NASAPerformance,NASAEffort,NASAFrustration,RawNASASum,RawNASA,UX1,UX2,GEQ_Competence,GEQ_Immersion,GEQ_PosAff
0,1,1,0,2,0,1,1,1,5,1,...,4,4,5,30,0.500000,4,4,1.0,0.5,1.0
1,1,6,0,2,0,0,1,2,5,1,...,2,8,5,27,0.450000,4,2,1.0,0.0,1.5
2,1,4,2,2,0,0,1,1,10,1,...,3,9,2,37,0.616667,4,4,2.0,0.0,1.0
3,1,2,3,1,0,0,1,2,6,1,...,2,4,4,24,0.400000,4,6,2.0,0.0,1.5
4,1,7,3,4,0,0,3,3,3,1,...,1,2,1,22,0.366667,4,2,3.5,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,20,7,3,3,2,3,3,3,10,13,...,4,11,13,57,0.950000,5,5,3.0,2.5,3.0
115,20,3,4,2,2,2,3,2,10,13,...,3,10,16,61,1.016667,3,5,3.0,2.0,2.5
116,20,2,4,3,2,2,3,3,10,16,...,4,10,17,63,1.050000,3,5,3.5,2.0,3.0
117,20,5,4,3,1,2,3,2,10,13,...,5,10,18,62,1.033333,5,5,3.5,1.5,2.5


In [52]:
dfQAll2.groupby("BlockNumber").UX2.describe()

,count,mean,std,min,25%,50%,75%,max
BlockNumber,,,,,,,,
1,17.0,3.823529,1.810915,1.0,3.0,4.0,5.0,7.0
2,17.0,3.882353,2.342070,1.0,1.0,5.0,6.0,7.0
3,17.0,3.882353,2.204607,1.0,2.0,4.0,5.0,7.0
4,17.0,4.235294,2.107410,1.0,2.0,5.0,6.0,7.0
5,17.0,4.117647,2.260596,1.0,2.0,5.0,6.0,7.0
6,17.0,3.647059,2.289683,1.0,1.0,4.0,6.0,7.0
7,17.0,3.705882,2.391775,1.0,1.0,4.0,5.0,7.0


In [56]:
df1 = dfQAll2[dfQAll2['BlockNumber']==6]
df2 = dfQAll2[dfQAll2['BlockNumber']==7]

In [63]:
from statistics import mean, stdev
from math import sqrt

var = "UX2"
cohens_d = (mean(df1[var]) - mean(df2[var])) / (sqrt((stdev(df1[var]) ** 2 + stdev(df2[var]) ** 2) / 2))
print(cohens_d)

-0.025124458419184
